In [2]:
from bs4 import BeautifulSoup
import pandas as pd
import requests
import re
import sqlite3
import time
import sys

In [10]:
Id = []
Title = []
Time_ = []
Type = []
Duration = []
ImgLink = []

movieDesc = []
Released = []
Genre = []
Casts = []
Country = []
Production = []
IMDB_rating = []
Cover_bg = []

# Scraping settings
type = "movie"
quality = "all"
year = "all"
country = "all"
genre = "all"

headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/111.0.0.0 Safari/537.36'}

# Start timer
start_time = time.time()

r1,r2 = 200 ,300
id = r1

total_pages = r2 -r1


for i in range(r1,r2):
    url = f"https://moviesjoy.is/filter?type={type}&quality={quality}&release_year={year}&genre={genre}&country={country}&page={i+1}"

    # Fetch the webpage
    webpage = requests.get(url, headers=headers).text
    soup = BeautifulSoup(webpage, 'lxml')
    
    # Extract movies
    movie = soup.find_all('div', class_='flw-item')
    
    # Time for each iteration
    iteration_start = time.time()

    for m in movie:
        
        try:

            Movie_Link = m.find('a').get('href')
            Movi_Url = f"https://moviesjoy.is{Movie_Link}"
            moviePage = requests.get(Movi_Url, headers=headers).text
            soup1 = BeautifulSoup(moviePage, 'lxml')
        except:
            continue

        Id.append(id)
        id = id+1
        Title.append(m.find('a').get('title'))
        Time_.append(m.find(class_='fdi-item').text.strip())
        Type.append(m.find(class_='fdi-type').text.strip())
        Duration.append(m.find(class_='fdi-duration').text.strip())
        ImgLink.append(m.find('div', class_='film-poster').find('img').get('data-src'))
        
        try:
            style = soup1.find('div', class_='cover_follow')['style']
            Cover_bg.append(re.search('url\((.*)\)', style)[1])
        except:
            Cover_bg.append("Na")

        try:
            movieDesc.append(soup1.find('div', class_='description').text.strip())        
        except:
            movieDesc.append("Na")
        
        try:
            IMDB_rating.append(re.split('IMDB:', soup1.find('button', class_='btn-imdb').text)[1].strip(' '))
        except:
            IMDB_rating.append("Na")
        
        data_div = str(soup1.find_all('div', class_='row-line'))

        try:
            Genre.append(re.findall('genre/([^"]+)', data_div))        
        except:
            Genre.append("Na")

        try:
            Casts.append(re.findall('cast/([^"]+)', data_div))        
        except:
            Casts.append("Na")

        try:
            Released.append(re.split('Released:\s</strong></span>([^<]+)', str(data_div))[1].strip(" ").strip('\n'))
        except:
            Released.append('Na')

        try:
            Production.append(re.findall('/production/([^"]+)', data_div)) 
        except: 
            Production.append('Na')
        
        try:
            Country.append(re.split('/country/([^"]+)', data_div)[1])
        except:
            Country.append("Na")
        
        # Progress indicator for each movie
        sys.stdout.write('_')
        sys.stdout.flush()
    
    # Calculate time taken for this iteration
    iteration_end = time.time()
    time_taken = iteration_end - iteration_start
    
    # Estimate total time remaining
    time_elapsed = time.time() - start_time
    estimated_total_time = (time_elapsed / (i -r1 + 1)) * total_pages
    estimated_remaining_time = estimated_total_time - time_elapsed

    # Print the progress
    sys.stdout.write(f"\nPage {i + 1}/{r2} completed. ETA : {int(estimated_remaining_time // 60)}m {int(estimated_remaining_time % 60)}s.\n")
    sys.stdout.flush()

# Total time taken
total_time_taken = time.time() - start_time
print(f"\nScraping completed in {int(total_time_taken // 60)} minutes {int(total_time_taken % 60)} seconds.")


________________________________
Page 201/300 completed. ETA : 30m 49s.
________________________________
Page 202/300 completed. ETA : 28m 32s.
________________________________
Page 203/300 completed. ETA : 28m 18s.
________________________________
Page 204/300 completed. ETA : 27m 28s.
________________________________
Page 205/300 completed. ETA : 27m 9s.
________________________________
Page 206/300 completed. ETA : 26m 57s.
________________________________
Page 207/300 completed. ETA : 26m 30s.
________________________________
Page 208/300 completed. ETA : 25m 58s.
_______________________________
Page 209/300 completed. ETA : 25m 33s.
________________________________
Page 210/300 completed. ETA : 25m 7s.
________________________________
Page 211/300 completed. ETA : 24m 44s.
________________________________
Page 212/300 completed. ETA : 24m 22s.
________________________________
Page 213/300 completed. ETA : 24m 6s.
________________________________
Page 214/300 completed. ETA : 23m 5

In [11]:
columns = ['Id','Title' ,'Time' ,'Type', 'Duration', 'Img' ,'CoverBg','movieDesc', 'Released' ,'Genre' ,'Casts' ,'Country', 'Production' ,'IMDB_rating']
pair = [pair for pair in zip(Id,Title ,Time_ ,Type ,Duration, ImgLink ,Cover_bg, movieDesc ,Released ,Genre ,Casts, Country ,Production ,IMDB_rating)]
df = pd.DataFrame(pair, columns=columns )

In [12]:
df.to_csv("movie.csv",index=False, header=False , mode='a')
# df.to_csv("movie.csv",index=False, header=True )
print("CSV file created successfully!")


CSV file created successfully!
